In [ ]:
!uv pip install --upgrade git+https://github.com/juspay/cada-rescript

In [ ]:
!uv pip install --upgrade git+https://github.com/juspay/codetraverse@convert-to-lib

In [ ]:
import codetraverse
codetraverse.create_fdep_data(root_dir="/Users/sakthi.n/Documents/Work/rescript-euler-dashboard", language="rescript")

In [ ]:
from rescript_ast_diff import generate_pr_changes_bitbucket
from rescript_ast_diff.bitbucket import BitBucket 

BASE_URL = "https://bitbucket.juspay.net/rest"
PROJECT_KEY = "JBIZ"
REPO_SLUG = "rescript-euler-dashboard"
AUTH = ("sakthi.n@juspay.in", "BBDC-NDg5ODgwNDM2MzkyOjgy8c70YxFmjQlfjSGQD4895tx5")
HEADERS = {"Accept": "application/json"}
# PR_ID = "22113"


print(PR_ID)
bitbucket = BitBucket(BASE_URL, PROJECT_KEY, REPO_SLUG, AUTH, HEADERS)
generate_pr_changes_bitbucket(PR_ID, bitbucket, quiet=True)

In [ ]:
from pydantic import BaseModel
from typing import Optional

class Data(BaseModel):
   moduleName: str
   addedFunctions: list = []
   modifiedFunctions: list = []
   deletedFunctions: list = []
   addedTypes: list = []
   modifiedTypes: list = []
   deletedTypes: list = []
   addedExternals: list = []
   modifiedExternals: list = []
   deletedExternals: list = []

import json

with open('./detailed_changes.json', 'r') as file:
    data = json.load(file)

filewisecomponents = []
for d in data:
    filewisecomponents.append(Data.model_validate(d))

In [ ]:
pull_request = bitbucket.get_pr_bitbucket(PR_ID)
latest_commit, old_commit = pull_request["fromRef"]["latestCommit"], pull_request["toRef"]["latestCommit"]

import json
response = bitbucket.get_changed_files_from_commits_raw(from_commit=latest_commit,to_commit=old_commit)

result = json.loads(response)

# with open("./gitdiff.json", "w") as f:
#     json.dump(result, f, indent=4)

In [ ]:
def get_codeBlock(lines,ok):  
    
    end,code = 0,"" 
    for line in lines:
        end = line['destination'] if ok else line['source']
        code += line['line']
    
    return (end,code)

added_changes = dict()
removed_changes = dict()

for files in result['diffs']:
    if files['source'] is None:
        continue
    filename = files['source']['components'][-1]
    for k in files['hunks']:
        for seg in k['segments']:
            if seg['type'] == 'ADDED':
                code = get_codeBlock(seg['lines'],1)

                if added_changes.get(filename,"") == "": 
                   added_changes[filename] = [code]
                else:
                   added_changes[filename].append(code)
            elif seg['type'] == 'REMOVED':
                code = get_codeBlock(seg['lines'],0)
                if removed_changes.get(filename,"") == "": 
                    removed_changes[filename] = [code] 
                else: 
                    removed_changes[filename].append(code)


In [ ]:
component_wise_sys_prompt = """
You are a QA automation assistant. You will receive a frontend function or component, along with metadata:
- Type of change (ADDED, MODIFIED, or DELETED)
- A git diff (only for MODIFIED)
- A list of parent components (that use this one)
- A list of child components (used within this one)

Your job is to generate **only user-level test cases** — that is, tests based on what the end user would see or interact with — regardless of whether the input is a component or a function.

Key Guidelines:

1. Treat all code — whether function or component — as potentially impacting the user interface.
   - If it is a **function**, consider how its logic affects UI behavior (e.g., rendering conditions, data shown, button enabled/disabled, etc.).
   - Do not write low-level unit tests. Focus on observable user outcomes.
   
2. Interpret changes based on the type:
   - ADDED → Cover new user interactions or behavior this introduces.
   - MODIFIED → Cover only user-visible changes based on the git diff.
   - DELETED → Skip unless the deletion breaks a user flow or visible behavior.

3. Use **parent components** to understand how this code is used in real UI contexts.
4. Use **child components** to identify if their behavior contributes to user experience changes through this component.
5. Do **not** write tests for parent or child components themselves — only for this component and its visible behavior as used in the app.
6. If a MODIFIED component removes logic (e.g., API calls, props), and that logic previously enabled behavior in a child component:
   - You must generate test cases that verify the downstream impact on visible user interactions.
   - Do this even if the child component’s code is not changed — because the current component is responsible for supplying data or context.
   - For example, if an API call is removed and that breaks a search feature in a child component, include a test case like “Verify that search in [child] continues to work as expected”.

Output Format:
Only return a **valid JSON array** like this:
[
  {
    "testCaseId": "TC001",
    "description": "Brief summary of what the test verifies",
    "stepsToReproduce": [
      "Step 1",
      "Step 2"
    ],
    "expectedResult": "Expected behavior"
  }
]

Do not include any explanation or wrap it in markdown.
Only return the JSON.
Every test case should be clear, relevant, user-focused, and verifiable.
"""

component_wise_prompt = """
You are given a single frontend function or component from a pull request.

You are also told whether the component/function is newly ADDED, MODIFIED, or DELETED.
If it is MODIFIED, a git diff is also provided.
Additionally, a list of:
- Parent components (that use this one)
- Child components (used within this one)
is provided to give full usage context.

Your task is to generate **only user-level test cases** that reflect how changes in the code affect **visible UI behavior** or **user interactions**.

Return only **valid JSON** like this:

[
  {{
    "testCaseId": "TC001",
    "description": "Brief summary of what the test verifies",
    "stepsToReproduce": [
      "Step 1",
      "Step 2",
      "..."
    ],
    "expectedResult": "Expected behavior"
  }}
]

Guidelines:

- Treat functions as user-facing: consider how logic inside a function affects what the user sees or interacts with.
- For ADDED → cover all visible behavior or interactions introduced.
- For MODIFIED → cover only the changed user-facing behavior.
- For DELETED → skip unless user flow is broken by the deletion.
- Use parent and child components for context — how the current component is used and what it includes — but only test the current one.

Input:
Component or Function Code:
{component_or_function_code}

Change Type:
{change_type} 

Git Diff (if MODIFIED):
{git_diff}

Used By (Parent Components):
{parent_component_list}

Uses (Child Components):
{child_component_list}
"""



In [ ]:
from codetraverse.path import load_graph
import os 

graph_path = os.getcwd() +  "/graph/repo_function_calls.gpickle"
graph = load_graph(graph_path)

In [ ]:
from codetraverse.path import find_path

def get_relevant_nodes(currcomp):

    print("currCompon: ",currcomp)
    parent_code,child_code = "",""
    
    try:
        resp = find_path(graph_path,currcomp,quiet=True)
        for par in resp[0]:
            node = graph.nodes.get(par,None)
            parent_code += "Component Name: " + par + "\n Component Code: " + node.get("code","NA") + "\n"
            # print(node)
        
        for child in resp[2]:
            if "::" in child:
                node = graph.nodes.get(child, None)
                if node.get('category',"") in ['module','function']:
                    # print(node)
                    child_code += "Component Name: " + child + "\n Component Code: " + node.get("code","NA") + "\n" 

        return (parent_code,child_code)
    except Exception as ex:
        print(currcomp + "Error: " + ex)

In [ ]:
for res in filewisecomponents:
    moduleName = os.path.basename(res.moduleName)
    nodeName = os.path.basename(res.moduleName).split('.')[0]
    ls = []
    for fun in res.addedFunctions:
        name,code,metadata = fun 
        content = (name + "\n" + code).strip()
        
        if name == "make" or "::" not in name:
            name = nodeName + "::" + name
        parent_comps,child_comps = get_relevant_nodes(name)
        
        utils = metadata | {"component_or_function_code": content,"parent_component_list": parent_comps,"child_component_list": child_comps,"change_type": "ADDED","git_diff":""}
        fun[-1] = utils

        print(utils)
        ls.append(fun) 
    res.addedFunctions = ls 

    ls = []
    for fun in res.deletedFunctions:
        name,code,metadata = fun 
        content = (name + "\n" + code).strip()

        if name == "make" or "::" not in name:
            name = nodeName + "::" + name

        utils =  metadata | {"component_or_function_code": content,"parent_component_list": "NA","child_component_list": "NA","change_type": "REMOVED","git_diff":""}
        fun[-1] = utils
        ls.append(fun)
    res.deletedFunctions = ls 

    ls = [] 
    for fun in res.modifiedFunctions:
        name,old_code,new_code,metadata = fun 
        start = metadata['new_start'][0] 
        end = metadata['new_end'][0]

        added_pieces = ""
        for (pos,code) in added_changes.get(moduleName,[]):
            if pos >= start and pos <= end:
                added_pieces += "+" + code + "\n"

        removed_pieces = "" 
        for (pos,code) in removed_changes.get(moduleName,[]):
            if pos >= start and pos <= end:
                removed_pieces += "-" + code + "\n"

        content = (name + "\n" + old_code).strip()
        git_diff = added_pieces + "\n" + removed_pieces

        if name == "make" or "::" not in name:
            name = nodeName + "::" + name
        parent_comps,child_comps = get_relevant_nodes(name)

        utils = metadata | {"component_or_function_code": content,"parent_component_list": parent_comps,"child_component_list": child_comps,"change_type": "MODIFIED","git_diff":git_diff}
        fun[-1] = utils
        print(utils)
        ls.append(fun)
    res.modifiedFunctions = ls 


In [ ]:
json_list = [] 

for res in filewisecomponents:
    json_list.append(res.model_dump())

with open("./unit_testing.json", "w") as f:
    json.dump(json_list, f, indent=4)

In [ ]:
AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
AWS_SECRET_KEY = os.getenv('AWS_SECRET_KEY')
BUCKET_NAME = os.getenv('BUCKET_NAME', "euler-jenkins-assets")
S3_FOLDER_NAME = os.getenv("S3_FOLDER_NAME",'manifest')
LOCAL_BUCKET_NAME = os.getenv("BUCKET_NAME", "bitbot-test")
LOCAL_AWS_REGION = os.getenv("AWS_REGION", "us-east-1")
LOCAL_AWS_ENDPOINT = os.getenv("AWS_ENDPOINT_URL", "http://127.0.0.1:4566")
LOCAL_AWS_PROFILE = os.getenv("AWS_PROFILE", "localstack")
LOCAL_TESTING = os.getenv("LOCAL_TESTING",True)

def normalize_file_path(file_name:str):
    if file_name.startswith("./"):
        return file_name[2:]
    if file_name.startswith("/"):
        return file_name[1:]
    else: return file_name

In [ ]:
import boto3

def upload_to_s3(file_name, object_name=None):

    n_file_name = normalize_file_path(file_name)
    if object_name is None:
        object_name = f"manifest/{latest_commit + "_" + n_file_name}"
    if not LOCAL_TESTING:
        s3_client = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_KEY)
    else:
        boto3.setup_default_session(profile_name = LOCAL_AWS_PROFILE)
        s3_client = boto3.client('s3', region_name=LOCAL_AWS_REGION, endpoint_url=LOCAL_AWS_ENDPOINT, aws_secret_access_key='test', aws_access_key_id='test')
    try:

        if LOCAL_TESTING:
            s3_client.upload_file(file_name, LOCAL_BUCKET_NAME, object_name)
            return
        s3_client.upload_file(file_name, LOCAL_BUCKET_NAME, object_name)
        
    except Exception as e:
        raise e


upload_to_s3("./unit_testing.json")